# Regression Week 1: Simple Linear Regression

In this notebook we will use data on house sales in King County to predict house prices using simple (one input) linear regression. You will:
* Use Turi Create SArray and SFrame functions to compute important summary statistics
* Write a function to compute the Simple Linear Regression weights using the closed form solution
* Write a function to make predictions of the output given the input feature
* Turn the regression around to predict the input given the output
* Compare two different models for predicting house prices

In this notebook you will be provided with some already complete code as well as some code that you should complete yourself in order to answer quiz questions. The code we provide to complete is optional and is there to assist you with solving the problems but feel free to ignore the helper code and write your own.

# Fire up Pandas and NumPy

In [1]:
import pandas as pd
import numpy as np

# Load house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [3]:
types = {
    "bathrooms": float,
    "waterfront": int,
    "sqft_above": int,
    "sqft_living15": float,
    "grade": int,
    "yr_renovated": int,
    "price": float,
    "bedrooms": float,
    "zipcode": str,
    "long": float,
    "sqft_lot15": float,
    "sqft_living": float,
    "floors": str,
    "condition": int,
    "lat": float,
    "date": str,
    "sqft_basement": int,
    "yr_built": int,
    "id": str,
    "sqft_lot": int,
    "view": int,
}
sales = pd.read_csv("Data/kc_house_data.csv", parse_dates=["date"], dtype=types)

In [4]:
sales.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,2014-10-13,221900.0,3.0,1.00,1180.0,5650,1,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340.0,5650.0
1,6414100192,2014-12-09,538000.0,3.0,2.25,2570.0,7242,2,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690.0,7639.0
2,5631500400,2015-02-25,180000.0,2.0,1.00,770.0,10000,1,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720.0,8062.0
3,2487200875,2014-12-09,604000.0,4.0,3.00,1960.0,5000,1,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360.0,5000.0
4,1954400510,2015-02-18,510000.0,3.0,2.00,1680.0,8080,1,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800.0,7503.0


# Split data into training and testing

We use seed=0 so that everyone running this notebook gets the same results.  In practice, you may set a random seed (or let Turi Create pick a random seed for you).  

In [5]:
train = pd.read_csv("Data/kc_house_train_data.csv", parse_dates=['date'], dtype=types)
test = pd.read_csv("Data/kc_house_test_data.csv", parse_dates=['date'], dtype=types)

In [6]:
len(train), len(test)

(17384, 4229)

# Useful SFrame summary functions

In order to make use of the closed form solution as well as take advantage of turi create's built in functions we will review some important ones. In particular:
* Computing the sum of an SArray
* Computing the arithmetic average (mean) of an SArray
* multiplying SArrays by constants
* multiplying SArrays by other SArrays

In [8]:
# Let's compute the mean of the House Prices in King County in 2 different ways.
prices = sales.price
print(f"average price via method 1: {prices.sum() / len(prices)}")
print(f"average price via method 2: {prices.mean()}")

average price via method 1: 540088.1417665294
average price via method 2: 540088.1417665294


# Build a generic simple linear regression function 

Complete the following function to compute the simple linear regression slope and intercept:

In [10]:
def simple_linear_regression(input_feature, output):
    """Using closed form: normal equation apporach"""
    
    num = len(input_feature)
    # compute the sum of input_feature and output
    sum_input, sum_output = input_feature.sum(), output.sum()

    # compute the product of the output and the input_feature and its sum
    sum_input_output = np.sum(input_feature * output)

    # compute the squared value of the input_feature and its sum
    sum_square_input = np.sum(input_feature * input_feature)

    # use the formula for the slope
    nominator = sum_input_output - ((sum_input * sum_output) / num)
    denominator = sum_square_input - ((sum_input * sum_input) / num)
    slope = nominator / denominator

    # use the formula for the intercept
    intercept = sum_output / num - slope * (sum_input / num)

    return (intercept, slope)

We can test that our function works by passing it something where we know the answer. In particular we can generate a feature and then put the output exactly on a line: output = 1 + 1\*input_feature then we know both our slope and intercept should be 1

In [19]:
test_feature = pd.Series(range(5))

test_output = 1 + 1 * test_feature

intercept, slope = simple_linear_regression(test_feature, test_output)

print(f"Intercept: {intercept}")
print(f"Slope: {slope}")

Intercept: 1.0
Slope: 1.0


Now that we know it works let's build a regression model for predicting price based on sqft_living. Rembember that we train on train_data!

In [12]:
sqft_intercept, sqft_slope = simple_linear_regression(train.sqft_living, train.price)
print(f"Intercept: {sqft_intercept}")
print(f"Slope: {sqft_slope}")

Intercept: -47116.07907289418
Slope: 281.9588396303426


# Predicting Values

Now that we have the model parameters: intercept & slope we can make predictions. Using SArrays it's easy to multiply an SArray by a constant and add a constant value. Complete the following function to return the predicted output given the input_feature, slope and intercept:

In [14]:
def get_regression_predictions(input_feature, intercept, slope):
    predicted_values = intercept + slope * input_feature
    return predicted_values

Now that we can calculate a prediction given the slope and intercept let's make a prediction. Use (or alter) the following to find out the estimated price for a house with 2650 squarefeet according to the squarefeet model we estiamted above.

**Quiz Question: Using your Slope and Intercept from (4), What is the predicted price for a house with 2650 sqft?**

In [15]:
my_house_sqft = 2650
estimated_price = get_regression_predictions(my_house_sqft, sqft_intercept, sqft_slope)
print(
    f"The estimated price for a house with {my_house_sqft} squarefeet is ${estimated_price:.2f}"
)

The estimated price for a house with 2650 squarefeet is $700074.85


# Residual Sum of Squares

Now that we have a model and can make predictions let's evaluate our model using Residual Sum of Squares (RSS). Recall that RSS is the sum of the squares of the residuals and the residuals is just a fancy word for the difference between the predicted output and the true output. 

Complete the following (or write your own) function to compute the RSS of a simple linear regression model given the input_feature, output, intercept and slope:

In [17]:
def get_residual_sum_of_squares(input_feature, output, intercept, slope):
    # First get the predictions
    predict = get_regression_predictions(input_feature, intercept, slope)
    errors = output - predict
    return np.sum(errors ** 2)

Let's test our get_residual_sum_of_squares function by applying it to the test model where the data lie exactly on a line. Since they lie exactly on a line the residual sum of squares should be zero!

In [ ]:
print(get_residual_sum_of_squares(
    test_feature,
    test_output,
    test_intercept,
    test_slope)
 ) # should be 0.0

Now use your function to calculate the RSS on training data from the squarefeet model calculated above.

**Quiz Question: According to this function and the slope and intercept from the squarefeet model What is the RSS for the simple linear regression using squarefeet to predict prices on TRAINING data?**

In [98]:
rss_prices_on_sqft = get_residual_sum_of_squares(
    train.sqft_living,
    train.price,
    sqft_intercept,
    sqft_slope
)
print(f'The RSS of predicting Prices based on Square Feet is : {rss_prices_on_sqft}')

The RSS of predicting Prices based on Square Feet is : 1201918354177283.0


# Predict the squarefeet given price

What if we want to predict the squarefoot given the price? Since we have an equation y = a + b\*x we can solve the function for x. So that if we have the intercept (a) and the slope (b) and the price (y) we can solve for the estimated squarefeet (x).

Complete the following function to compute the inverse regression estimate, i.e. predict the input_feature given the output.

In [112]:
def inverse_regression_predictions(output, intercept, slope):
    return (output - intercept) / slope

Now that we have a function to compute the squarefeet given the price from our simple regression model let's see how big we might expect a house that costs $800,000 to be.

**Quiz Question: According to this function and the regression slope and intercept from (3) what is the estimated square-feet for a house costing $800,000?**

In [100]:
my_house_price = 800000
estimated_squarefeet = inverse_regression_predictions(
    my_house_price, sqft_intercept, sqft_slope
)
print(
    f"The estimated squarefeet for a house worth ${my_house_price:.2f} is {estimated_squarefeet:.0f}"
)

The estimated squarefeet for a house worth $800000.00 is 3004


# New Model: estimate prices from bedrooms

We have made one model for predicting house prices using squarefeet, but there are many other features in the sales SFrame. 
Use your simple linear regression function to estimate the regression parameters from predicting Prices based on number of bedrooms. Use the training data!

In [86]:
# Estimate the slope and intercept for predicting 'price' based on 'bedrooms'
bedrooms_intercept, bedrooms_slope = simple_linear_regression(train.bedrooms, train.price)

In [88]:
bedrooms_intercept, bedrooms_slope

(109473.1776229596, 127588.95293398784)

# Test your Linear Regression Algorithm

Now we have two models for predicting the price of a house. How do we know which one is better? Calculate the RSS on the TEST data (remember this data wasn't involved in learning the model). Compute the RSS from predicting prices using bedrooms and from predicting prices using squarefeet.

**Quiz Question: Which model (square feet or bedrooms) has lowest RSS on TEST data? Think about why this might be the case.**

In [96]:
# Compute RSS when using bedrooms on TEST data:
get_residual_sum_of_squares(
    test.bedrooms,
    test.price,
    bedrooms_intercept,
    bedrooms_slope
)

493364585960300.9

In [95]:
# Compute RSS when using squarefeet on TEST data:
get_residual_sum_of_squares(
    test.sqft_living,
    test.price,
    sqft_intercept,
    sqft_slope
)

275402933617812.12

In [111]:
pd.DataFrame(
    [493364585960300, 275402933617812],
    index=['Bedrooms', 'sqft_living'],
    columns=['RSS'],
).sort_values(by='RSS', ascending=False)

,RSS
Bedrooms,493364585960300
sqft_living,275402933617812


* Clearly, the `sqft_living` is a better than number of bedrooms.